# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier as T
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import MultinomialNB as bayes
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.datasets import make_classification

from razdel import tokenize as rt

In [ ]:
data = pd.read_csv('labeled.csv')

In [ ]:
def razdel_tokenizer(text):
    return [token.text for token in list(rt(text))]

In [ ]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
vectorizer1 = TfidfVectorizer (tokenizer = razdel_tokenizer)
X1 = vectorizer1.fit_transform(train.comment)
X1_test = vectorizer1.transform(test.comment)
vectorizer1.get_feature_names_out()

In [ ]:
vectorizer2 = TfidfVectorizer () #tokenizer = default
X2 = vectorizer2.fit_transform(train.comment)
X2_test = vectorizer2.transform(test.comment)
vectorizer2.get_feature_names_out()

In [ ]:
y1 = train.toxic.values
y1_test = test.toxic.values
y2 = train.toxic.values
y2_test = test.toxic.values

In [ ]:
clf = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clf.fit(X1, y1)
preds1 = clf.predict(X1_test)
print(classification_report(y1_test, preds1, zero_division=0))

In [ ]:
clf = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clf.fit(X2, y2)
preds2 = clf.predict(X2_test)
print(classification_report(y2_test, preds2, zero_division=0))

Встроенный токенизатор побеждает

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

*random_seed не считается за параметр

In [ ]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [ ]:
vectorizerL = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_L = vectorizerL.fit_transform(train.comment)
X_for_L_test = vectorizerL.transform(test.comment)
#vectorizerTF.get_feature_names_out()

In [ ]:
vectorizerB = CountVectorizer (min_df=4, max_df=0.3, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_B = vectorizerB.fit_transform(train.comment)
X_for_B_test = vectorizerB.transform(test.comment)
#vectorizerC.get_feature_names_out()

In [ ]:
yL = train.toxic.values
yL_test = test.toxic.values
yB = train.toxic.values
yB_test = test.toxic.values

In [ ]:
clfL = LogisticRegression(penalty='l2', class_weight= 'balanced')
clfL.fit(X_for_L, yL)
predsL = clfL.predict(X_for_L_test)
print(classification_report(yL_test, predsL, zero_division=0))

In [ ]:
clfB = bayes(alpha=1, force_alpha=True)
clfB.fit(X_for_B, yB)
predsB = clfB.predict(X_for_B_test)
print(classification_report(yB_test, predsB, zero_division=0))

In [ ]:
predsB_proba = clfB.predict_proba(X_for_B_test)
predsL_proba = clfL.predict_proba(X_for_L_test)

In [ ]:
from collections import Counter
from pprint import pprint

In [ ]:
counterB = Counter({comm: prob[1] for comm, prob in zip(data.comment, predsB_proba)})
B_fin = counterB.most_common(10)
pprint(B_fin)

In [ ]:
counterL = Counter({comm: prob[1] for comm, prob in zip(data.comment, predsL_proba)})
L_fin = counterL.most_common(10)
pprint(L_fin)

In [ ]:
same = list()
for i in L_fin:
    for j in B_fin:
        if i==j and j not in same:
            same.append(j)
same

одинаковых текстов нет, тексты действительно токсичные

## Задание 3 (4 балла - 1 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.75

In [62]:
counter_tB = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerB.vocabulary_,
            clfB.feature_log_prob_[1]
        )}
)
counter_tB.most_common(5)

[('доллары', np.float64(-3.9107735045537746)),
 ('узнает', np.float64(-5.0052999856993035)),
 ('пар', np.float64(-5.429284141574706)),
 ('разбить', np.float64(-5.791313850651098)),
 ('стране', np.float64(-5.804827569817821))]

In [ ]:
counter_tL = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerB.vocabulary_,
            #здесь не знаю, что использовать, чтобы получить значения
        )}
)
counter_tL.most_common(5)

In [63]:
vectorizerRFC = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_RFC = vectorizerRFC.fit_transform(train.comment)
X_for_RFC_test = vectorizerRFC.transform(test.comment)

vectorizerT = TfidfVectorizer (min_df=4, max_df=0.1, max_features=10000, stop_words=russian_stopwords, ngram_range=(1, 3))
X_for_T = vectorizerT.fit_transform(train.comment)
X_for_T_test = vectorizerT.transform(test.comment)

In [64]:
yRFC = train.toxic.values
yRFC_test = test.toxic.values
yT = train.toxic.values
yT_test = test.toxic.values

In [65]:
clfRFC = RFC(min_samples_leaf = 2, class_weight = 'balanced')
clfRFC.fit(X_for_RFC, yRFC)
predsRFC = clfRFC.predict(X_for_RFC_test)

clfT = T(min_samples_leaf = 2, class_weight = 'balanced')
clfT.fit(X_for_T, yT)
predsT = clfT.predict(X_for_T_test)

In [66]:
counter_tRFC = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerRFC.vocabulary_,
            clfRFC.feature_importances_
        )}
)
counter_tRFC.most_common(5)

[('неё', np.float64(0.014246483998346498)),
 ('станцию', np.float64(0.011806632257174918)),
 ('передвижения', np.float64(0.009753879273825504)),
 ('образом', np.float64(0.009735116356764474)),
 ('говорит это', np.float64(0.00896039996890758))]

In [69]:
counter_tT = Counter({tok: feature
        for tok, feature
        in zip(
            vectorizerT.vocabulary_,
            clfT.feature_importances_
        )}
)
counter_tT.most_common(5)

[('неё', np.float64(0.017862057041721755)),
 ('передвижения', np.float64(0.01617632073210278)),
 ('90х', np.float64(0.015788618411920186)),
 ('станцию', np.float64(0.014617185990724991)),
 ('посмотрю', np.float64(0.012095070478296244))]